In [1]:
import numpy as np
#from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomTreesEmbedding

In [2]:
### Carregando arquivos de teste e treino ###

folder = "acmTesteTreino/"
arquivo_teste = "teste0.txt"
arquivo_treino = "treino0.txt"

file_treino = folder + arquivo_treino
file = open(file_treino, 'r', encoding="utf8")
treino = file.readlines()
file.close()

file_teste = folder + arquivo_teste
file = open(file_teste, 'r', encoding="utf8")
teste = file.readlines()
file.close()

In [3]:
n_termos = 59991

### Montando (X_treino, Y_treino) e (X_teste, Y_teste) ###

#Y_treino
Y_treino = []
for doc in treino:
    Y_treino.append(doc.split()[0])

#Y_teste
Y_teste = []
for doc in teste:
    Y_teste.append(doc.split()[0])

#X_treino
X_treino = np.zeros((len(treino), n_termos), dtype=np.int_)

for index,doc in enumerate(treino):    
    for termo in doc.split()[1:]:
        X_treino[index][int(termo.split(':')[0])] = int(termo.split(':')[1])
        
#X_teste
X_teste = np.zeros((len(teste), n_termos), dtype=np.int_)

for index,doc in enumerate(teste):    
    for termo in doc.split()[1:]:
        X_teste[index][int(termo.split(':')[0])] = int(termo.split(':')[1])


In [4]:
X_treino.shape, X_teste.shape

((19918, 59991), (4979, 59991))

In [10]:
### Carregando modelo em disco ###
from sklearn.externals import joblib
joblib_file = "encoder.pkl"
model_eforest = joblib.load(joblib_file)

In [18]:
### Definindo modelo encoder eForest ###
#model_eforest = RandomForestClassifier(n_estimators=300, max_depth=50, n_jobs=-1)
model_eforest = RandomTreesEmbedding(n_estimators=300, max_depth=None, n_jobs=-1)

In [19]:
### Treinando modelo encoder eForest ###
#model_eforest.fit(X_treino, Y_treino)
model_eforest.fit(X_treino)

RandomTreesEmbedding(max_depth=None, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
           random_state=None, sparse_output=True, verbose=0,
           warm_start=False)

In [20]:
### Salvando codificador em disco ###

from sklearn.externals import joblib
joblib_file = "encoder.pkl"
joblib.dump(model_eforest, joblib_file)

['encoder.pkl']

In [21]:
### Aanalisando árvores de decisão ###

k = 0 # árvore com menor número de folhas
altura_media = model_eforest.estimators_[0].tree_.max_depth # altura média das arvores
#n_folhas_medio = model_eforest.estimators_[0].tree_.get_n_leaves() # numero medio de folhas das arvores

for i in range(1,300):
    altura_media += model_eforest.estimators_[i].tree_.max_depth
    #n_folhas_medio += model_eforest.estimators_[i].tree_.get_n_leaves()
    #if model_eforest.estimators_[i].tree_.get_n_leaves() < model_eforest.estimators_[k].tree_.get_n_leaves():
    #    k = i        
altura_media = altura_media/300
#n_folhas_medio = n_folhas_medio/1000

print('Altura média: ' + str(altura_media))
#print('Média de número de folhas: ' + str(n_folhas_medio))
print('Altura da menor árvore: ' + str(model_eforest.estimators_[k].tree_.max_depth))

Altura média: 6270.5633333333335
Altura da menor árvore: 6337


In [9]:
### Visualizando árvore K  do codificador ###

from sklearn.tree import export_graphviz

tree = model_eforest.estimators_[k]

features = list(range(n_termos))
classes = ['0','1','2','3','4','5','6','7','8','9','10']

export_graphviz(tree, out_file='k_tree.dot', 
                feature_names = features,
                class_names = classes,
                rounded = True, proportion = False, 
                precision = 2, filled = True)

# Foi gerado o arquivo .dot
# Para converter em png execute o seguinte comando:
# dot k_tree.dot -Tpng -o k_tree.png

In [22]:
### Codificando a coleção de Treino ###
X_encode = model_eforest.encode(X_treino)
X_encode

array([[26320, 21484, 19994, ..., 19827, 12517, 16775],
       [18496, 28886, 32981, ..., 26442, 28634, 31957],
       [25901, 38207, 35433, ..., 19891, 32977, 19828],
       ...,
       [19403, 28545, 31361, ..., 12629, 28633, 36144],
       [ 6975, 19136,  6977, ..., 14901,  8102, 18168],
       [ 7207,  6567,  6387, ...,  9669,  7063, 10666]])

In [11]:
### Iniciando classificador SVM ###
from sklearn import svm

### Definindo o classificador SVM ###
model_svm = svm.SVC(kernel='linear', gamma=1)

In [ ]:
### Treinando modelo classificador SVM com a coleção Treino codificada (X_encode) ###
model_svm.fit(X_encode, Y_treino)

In [ ]:
#Score do modelo SVM
model_svm.score(X_encode, Y_treino)

In [ ]:
#Testando o classificador para a coleção Teste
X_encode_teste = model_eforest.encode(X_teste)
model_svm.score(X_encode_teste, Y_teste)